In [10]:
import pandas as pd
import numpy as np
from datetime import datetime
from tqdm import tqdm

In [11]:
#importing the data
age_data = pd.read_csv("age_data.csv")
tenure_data = pd.read_csv("tenure_data.csv")

/Users/nicoubide/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (0,2) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


First we're going to clean the age data

In [12]:
#Creating NaN consitency
age_data.fillna("NaN", inplace=True)
age_data["DOB"].replace("n.a.", "NaN", inplace=True)

In [13]:
#Creating a function that will get the "DOB" column in the format we want. "DOB" column has
#4 different types of inputs

#Type 1: 01 Apr 1918
#Type 2: Sep 1945
#Type 3: "1945"
#Type 4: 1945
#Type 5: "NaN"

def correct_dob(date):
    if date == "NaN":
        return date
    elif type(date) == int:
        return pd.to_datetime(date, format="%Y").strftime("%Y%m%d")
    elif len(date.split(" ")) == 1:
        return pd.to_datetime(date, format="%Y").strftime("%Y%m%d")
    elif len(date.split(" ")) == 2:
        return pd.to_datetime(date, format="%b %Y").strftime("%Y%m%d")
    elif len(date.split(" ")) == 3:
        return pd.to_datetime(date, format="%d %b %Y").strftime("%Y%m%d")
    else:
        return "ERROR"

dob_vector = np.vectorize(correct_dob)

In [14]:
#Updating the DOB column
age_data["DOB"] = dob_vector(age_data["DOB"])

In [15]:
#Age NANs are being left to be dealt with later

Now we're cleaning the tenure data

In [16]:
#Dropping the entries that either have an "N" in the StartDate or EndDate
n_list = list(np.where(np.logical_or(tenure_data["DateEndRole"]=="N",tenure_data["DateStartRole"]=="N"))[0])
tenure_data.drop(n_list, inplace=True)

In [17]:
#Helper function to update "C" entry to a value of "-1"
def update_end(date):
    if date == "C":
        return -1
    else:
        return pd.to_datetime(date, format="%Y%m%d")

end_vector = np.vectorize(update_end)

In [18]:
#Updating the "C" entry and converting start and end dates to datetime format
tenure_data["date_end_role"] = end_vector(tenure_data["DateEndRole"])
tenure_data["date_start_role"] = pd.to_datetime(tenure_data["DateStartRole"], format="%Y%m%d")

Now we can join the two datasets

In [21]:
#Executing the inner join
joined_data = pd.merge(left=tenure_data, right=age_data)

Defining functions that will help us create the panel dataset

In [22]:
def years_column(date_range):
    years = date_range.strftime("%Y").tolist()
    return years

In [23]:
def months_column(date_range):
    months = date_range.strftime("%m").tolist()
    return months

In [24]:
def id_column(_id_, n):
    id_array = [_id_] * n
    return id_array

In [25]:
#Creating a function that will give us "age" in the desired format - Y.MMM
def age_calculator(DOB, date):
    months = np.datetime64(date, "M") - np.datetime64(DOB, "M")
    age = months / np.timedelta64(12,'M')
    return age

#Vectorizing
age_vector = np.vectorize(age_calculator, otypes=[np.float])

In [26]:
def age_column(DOB, date_range):
    #handling "NaN" value
    if DOB == "NaN":
        ages = ["NaN"] * len(date_range)
    else:
        dob = pd.period_range(start=DOB, periods=1 , freq='M').strftime("%Y-%m").tolist()[0]
        reference_dates = date_range.strftime("%Y-%m").tolist()
        ages = list(age_vector(dob, reference_dates))
    return ages

In [27]:
def make_panel(idx, row):
   
    #original df
    original_df = pd.DataFrame(row).transpose()

    #Declaring key variables
    start = row["date_start_role"]
    end = (pd.to_datetime('today') if row["date_end_role"] == -1 else row["date_end_role"])
    date_range = pd.period_range(start=start, end=end, freq='M')
    _id_ = row["DirectorID"]
    dob = row["DOB"]
    
    #Creating the columns
    # years = years_column(date_range)
    # months = months_column(date_range)
    ages = age_column(dob, date_range)
    ids = id_column(_id_, len(date_range))
    date = date_range
    
    #Creating the df to be appended
    new_df = pd.DataFrame([date,ages,ids]).transpose()
    new_df.columns = ["date", "age", "DirectorID"]
    
    #Executing the join
    joined_df = pd.merge(left=original_df, right=new_df)
    
    return joined_df

In [28]:
#30 minutes to execute
dfs = []

for idx, row in tqdm(joined_data.iterrows(), total=joined_data.shape[0]):
    curr_df = make_panel(idx, row)
    dfs.append(curr_df)
    
panel_data = pd.concat(dfs)

100%|██████████| 91049/91049 [23:16<00:00, 65.22it/s]  


In [29]:
def add_role_tenure(df):

    ''' 
    Calculate role_tenure by simply taking the current date minus the date_start_role
    '''
    
    df["role_tenure"] = -1
    tenure_list = []

    for start_date, current_date in tqdm(zip(df["date_start_role"], df["date"]), total=df.shape[0]):
        tenure_list.append(np.datetime64(current_date, "M") - np.datetime64(start_date, "M"))
    
    df["role_tenure"] = tenure_list / np.timedelta64(12,'M')

    return df

#Creating the role tenure for our panel_data
panel_data = add_role_tenure(panel_data)

100%|██████████| 7047297/7047297 [03:26<00:00, 34097.71it/s]


In [30]:
panel_data = panel_data.sort_values(by=["CompanyName","DirectorName", "date"])

In [35]:
panel_data.to_csv("directorships.csv")

In [31]:
costco_data = panel_data[panel_data["CompanyName"].str.contains("Costco")].iloc[190:230]

In [32]:
costco_data.reset_index(inplace=True)

In [34]:
costco_data.to_csv("costco_data.csv")

In [ ]:
#Only keeping CEOs - RUN THIS WHEN COMPLETELY DONE
tenure_data = tenure_data[tenure_data["RoleName"].str.contains("CEO")]